In [2]:
from getpass import getpass
openai_api_key = getpass()

In [2]:
import os
import shutil
if os.path.exists("langchain"):
    shutil.rmtree("langchain")
os.system("git clone https://github.com/hwchase17/langchain.git --branch master --single-branch")

Cloning into 'langchain'...


0

In [3]:
from typing import Any, Dict, List, Optional, Union
from langchain.docstore.document import Document
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PythonLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers.language import LanguageParser
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [45]:
loader = GenericLoader.from_filesystem(
    "langchain/langchain/",
    glob="**/*.py",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON),
    show_progress=True
)
source_code_doc = loader.load()

/mnt/data/workspace/ai_projects/GenerativeApp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/865 [00:00<?, ?it/s]

100%|██████████| 865/865 [00:03<00:00, 242.57it/s]


In [55]:
source_code_doc[10].metadata

{'source': 'langchain/langchain/cache.py',
 'content_type': 'functions_classes',
 'language': <Language.PYTHON: 'python'>}

In [11]:
langchain_python_doc_loader = DirectoryLoader('langchain', glob="**/*.mdx", 
                                               loader_cls=TextLoader,
                                               show_progress=True, use_multithreading=True)
langchain_doc = langchain_python_doc_loader.load()

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = []
for markdown_document in langchain_doc:
    md_header_splits += markdown_splitter.split_text(markdown_document.page_content)
md_header_splits


  0%|          | 0/334 [00:00<?, ?it/s]

100%|██████████| 334/334 [00:00<00:00, 2551.67it/s]


[Document(page_content='By default, `PromptTemplate` will validate the `template` string by checking whether the `input_variables` match the variables defined in `template`. You can disable this behavior by setting `validate_template` to `False`  \n```python\ntemplate = "I am learning langchain because {reason}."  \nprompt_template = PromptTemplate(template=template,\ninput_variables=["reason", "foo"]) # ValueError due to extra variables\nprompt_template = PromptTemplate(template=template,\ninput_variables=["reason", "foo"],\nvalidate_template=False) # No error\n```', metadata={'Header 1': 'Validate template'}),
 Document(page_content='LangChain provides different types of `MessagePromptTemplate`. The most commonly used are `AIMessagePromptTemplate`, `SystemMessagePromptTemplate` and `HumanMessagePromptTemplate`, which create an AI message, system message and human message respectively.  \nHowever, in cases where the chat model supports taking chat message with arbitrary role, you can 

In [5]:
langchain_doc[0].page_content

'# Types of `MessagePromptTemplate`\n\nLangChain provides different types of `MessagePromptTemplate`. The most commonly used are `AIMessagePromptTemplate`, `SystemMessagePromptTemplate` and `HumanMessagePromptTemplate`, which create an AI message, system message and human message respectively.\n\nHowever, in cases where the chat model supports taking chat message with arbitrary role, you can use `ChatMessagePromptTemplate`, which allows user to specify the role name.\n\n\n```python\nfrom langchain.prompts import ChatMessagePromptTemplate\n\nprompt = "May the {subject} be with you"\n\nchat_message_prompt = ChatMessagePromptTemplate.from_template(role="Jedi", template=prompt)\nchat_message_prompt.format(subject="force")\n```\n\n<CodeOutputBlock lang="python">\n\n```\n    ChatMessage(content=\'May the force be with you\', additional_kwargs={}, role=\'Jedi\')\n```\n\n</CodeOutputBlock>\n\nLangChain also provides `MessagesPlaceholder`, which gives you full control of what messages to be ren

In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [6]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def num_tokens_from_string(docs: List[Document], encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = 0
    max_tokens = 0
    for doc in docs:
        if len(encoding.encode(doc.page_content)) > max_tokens:
            max_tokens = len(encoding.encode(doc.page_content))
        num_tokens += len(encoding.encode(doc.page_content))
    return num_tokens, max_tokens

In [24]:
token, max_token = num_tokens_from_string(source_code_doc, "cl100k_base")
print(f"Number of tokens in source code: {token}, max tokens in source code: {max_token}, price: {token * 0.0001/1000}")

Number of tokens in source code: 800960, max tokens in source code: 6667, price: 0.080096


In [12]:
token, max_token = num_tokens_from_string(md_header_splits, "cl100k_base")
print(f"Number of tokens in source code: {token}, max tokens in source code: {max_token}, price: {token * 0.0001/1000}")

Number of tokens in source code: 194615, max tokens in source code: 7692, price: 0.0194615


In [18]:
embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
# save to disk
vectorstore = Chroma.from_documents(documents=langchain_doc,embedding=embeddings_model, persist_directory="./chroma_db")
vectorstore.persist()

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID e074983f109dae3f46e6aa30adc1c34f in your email.) {
  "error": {
    "message": "The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID e074983f109dae3f46e6aa30adc1c34f in your email.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID e074983f10

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
question="How to use the MultiQueryRetriever?"
num_queries=3
llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(),llm=llm)

In [42]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

5

In [43]:
unique_docs

[Document(page_content='# Self-querying\n\nA self-querying retriever is one that, as the name suggests, has the ability to query itself. Specifically, given any natural language query, the retriever uses a query-constructing LLM chain to write a structured query and then applies that structured query to it\'s underlying VectorStore. This allows the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documented, but to also extract filters from the user query on the metadata of stored documents and to execute those filters.\n\n![](https://drive.google.com/uc?id=1OQUN-0MJcDUxmPXofgS7MqReEs720pqS)\n\nimport Example from "@snippets/modules/data_connection/retrievers/self_query/get_started.mdx"\n\n<Example/>\n', metadata={'source': 'langchain/docs/docs_skeleton/docs/modules/data_connection/retrievers/how_to/self_query/index.mdx'}),
 Document(page_content='---\nsidebar_position: 4\n---\n# Retrievers\n\nA retriever is an interface that

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0, openai_api_key=openai_api_key, max_token=2000), vectorstore.as_retriever())

WARNING! max_token is not default parameter.
                    max_token was transferred to model_kwargs.
                    Please confirm that max_token is what you intended.


NameError: name 'vectorstore' is not defined

In [41]:
chat_history = []
query = "Tell me how to implement a chatbot in python"
result = qa({"question": query, "chat_history": chat_history})
result

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5707 tokens (5451 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

# Try generating code from simple codebase

In [19]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    Language,
)

In [27]:
# You can also see the separators used for a given language
RecursiveCharacterTextSplitter.get_separators_for_language(Language.PYTHON)

['\nclass ', '\ndef ', '\n\tdef ', '\n\n', '\n', ' ', '']

In [39]:
with open("codebase/files.py", "r") as f:
    PYTHON_CODE = f.read()

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=1000, chunk_overlap=10
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

[Document(page_content='import os\nfrom typing import List\n\ndef combine_files(file_paths: List[str], combine_file_path: str):\n    [os.remove(file_path) for file_path in file_paths]\n    with open(combine_file_path, \'w\') as combine_file:\n        combine_file.write("I pranked you")\n\ndef remove_files_by_name(file_names: List[str], filter: str) -> List[str]:\n    return [file_name for file_name in file_names if filter in file_name]\n\ncombine_files(["test1.txt", "test1.txt"], "combined_file")', metadata={})]

In [40]:
embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
# save to disk
vectorstore_code = Chroma.from_documents(documents=python_docs,embedding=embeddings_model)

In [41]:
retriever = vectorstore_code.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 20
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 20

In [42]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_api_key)  # 'ada' 'gpt-3.5-turbo' 'gpt-4',
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [43]:
questions = [
    "I have 2 files (test1.txt and test2.txt), I'd like to combine them into one file (result.txt), how can i do it ?",
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

Number of requested results 20 is greater than number of elements in index 1, updating n_results = 1


-> **Question**: I have 2 files (test1.txt and test2.txt), I'd like to combine them into one file (result.txt), how can i do it ? 

**Answer**: You can use the `combine_files` function in the code provided. Pass the list of file paths ["test1.txt", "test2.txt"] as the first argument and the desired combined file path ("result.txt") as the second argument. The function will combine the contents of the two files into the specified combined file. 



In [ ]:
questions = [
    "I have 2 files (test1.txt and test2.txt and file.py), I'd like to filter the files containing test in it, how can I do that ?",
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

# Let's try to give the LLM the most recet version of streamlit in mind

In [4]:
from typing import Any, Dict, List, Optional, Union
from langchain.docstore.document import Document
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PythonLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers.language import LanguageParser
from langchain.text_splitter import MarkdownHeaderTextSplitter
import os
import shutil

In [5]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def num_tokens_from_string(docs: List[Document], encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = 0
    max_tokens = 0
    for doc in docs:
        if len(encoding.encode(doc.page_content)) > max_tokens:
            max_tokens = len(encoding.encode(doc.page_content))
        num_tokens += len(encoding.encode(doc.page_content))
    return num_tokens, max_tokens

In [16]:
if os.path.exists("streamlit"):
    shutil.rmtree("streamlit")
os.system("git clone https://github.com/streamlit/streamlit.git --branch master --single-branch")

Cloning into 'streamlit'...
Updating files: 100% (2205/2205), done.


0

In [127]:
loader = GenericLoader.from_filesystem(
    "streamlit/",
    glob="lib/streamlit/elements/**/*.py",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON),
    show_progress=True
)
streamlit_source_code_doc = loader.load()

token, max_token = num_tokens_from_string(streamlit_source_code_doc, "cl100k_base")
print(f"Number of tokens in source code: {token}, max tokens in source code: {max_token}, price: {token * 0.0001/1000}")

 28%|██▊       | 14/50 [00:00<00:00, 123.57it/s]

100%|██████████| 50/50 [00:00<00:00, 166.70it/s]


Number of tokens in source code: 100208, max tokens in source code: 4703, price: 0.010020800000000002


In [128]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=250, chunk_overlap=50
)
streamlit_source_code_doc = python_splitter.split_documents(streamlit_source_code_doc)

token, max_token = num_tokens_from_string(streamlit_source_code_doc, "cl100k_base")
print(f"Number of tokens in source code: {token}, max tokens in source code: {max_token}, price: {token * 0.0001/1000}")

Number of tokens in source code: 102761, max tokens in source code: 101, price: 0.010276100000000002


In [129]:
embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore_streamlit_code = Chroma.from_documents(documents=streamlit_source_code_doc,embedding=embeddings_model)

In [130]:
streamlit_code_retriever = vectorstore_streamlit_code.as_retriever()
streamlit_code_retriever.search_kwargs["distance_metric"] = "cos"
streamlit_code_retriever.search_kwargs["fetch_k"] = 20
streamlit_code_retriever.search_kwargs["maximal_marginal_relevance"] = True
streamlit_code_retriever.search_kwargs["k"] = 10

In [131]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, ConversationChain
from langchain.memory import ConversationBufferMemory

model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_api_key)  # 'ada' 'gpt-3.5-turbo' 'gpt-4',
qa_over_streamlit_code = ConversationalRetrievalChain.from_llm(model, retriever=streamlit_code_retriever)

model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_api_key)  # 'ada' 'gpt-3.5-turbo' 'gpt-4',
simple_conversation = ConversationChain(llm=model, memory=ConversationBufferMemory())

In [60]:
from IPython.display import display, Markdown
output = simple_conversation.run("I'm looking for a way to create a button in streamlit")
display(Markdown(output))

Oh, Streamlit is a great choice for building interactive web applications! To create a button in Streamlit, you can use the `st.button()` function. Here's an example:

```python
import streamlit as st

button_clicked = st.button("Click me!")

if button_clicked:
    st.write("Button was clicked!")
```

In this example, `st.button()` creates a button with the label "Click me!". When the button is clicked, the `button_clicked` variable becomes `True`, and the message "Button was clicked!" is displayed using `st.write()`.

Let me know if you need any more help with Streamlit!

In [61]:
output = simple_conversation.run("What about a chart ?")
display(Markdown(output))

To create a chart in Streamlit, you can use the `st.line_chart()`, `st.area_chart()`, or `st.bar_chart()` functions. Here's an example using `st.line_chart()`:

```python
import streamlit as st
import pandas as pd

data = pd.DataFrame({
    'x': [1, 2, 3, 4, 5],
    'y': [10, 5, 8, 3, 6]
})

st.line_chart(data)
```

In this example, `st.line_chart()` creates a line chart using the data provided in a Pandas DataFrame. You can also use the `st.area_chart()` function to create an area chart or the `st.bar_chart()` function to create a bar chart.

Let me know if you have any more questions about creating charts in Streamlit!

In [62]:
output = simple_conversation.run("What is the most recent version of streamlit ?")
display(Markdown(output))

As of the time of this conversation, the most recent version of Streamlit is 0.94.1. However, please note that software versions are subject to change, so it's always a good idea to check the official Streamlit website or documentation for the most up-to-date information.

In [72]:
output = simple_conversation.run("How do I get camera inputs in a streamlit app ?")
display(Markdown(output))

To get camera inputs in a Streamlit app, you can use the OpenCV library in Python. Here are the steps you can follow:

1. Install the necessary libraries by running the following command in your terminal:
   ```
   pip install streamlit opencv-python-headless
   ```

2. Import the required modules in your Python script:
   ```python
   import streamlit as st
   import cv2
   ```

3. Create a function to capture video from the camera:
   ```python
   def capture_video():
       cap = cv2.VideoCapture(0)
       while True:
           ret, frame = cap.read()
           if not ret:
               break
           st.image(frame, channels="BGR")
       cap.release()
   ```

4. Use the `st.button` function to create a button in your Streamlit app:
   ```python
   if st.button('Start Camera'):
       capture_video()
   ```

5. Run your Streamlit app using the following command in your terminal:
   ```
   streamlit run your_app.py
   ```

When you run the app and click on the "Start Camera" button, it will capture video from your default camera and display it in the Streamlit app.

Note: Make sure your camera is connected and accessible by OpenCV. If you have multiple cameras connected, you can specify the camera index in `cap = cv2.VideoCapture(0)` to select the desired camera (e.g., `cap = cv2.VideoCapture(1)` for the second camera).

In [74]:
output = simple_conversation.run("Do you know the function camera_input() from streamlit ?")
display(Markdown(output))

Yes, I know the `camera_input()` function from Streamlit. It is not a built-in function in Streamlit, but it is a hypothetical function that you are asking about. As of my knowledge, Streamlit does not have a built-in function called `camera_input()`. However, you can create a function called `camera_input()` yourself using the steps I mentioned earlier to capture video from a camera in a Streamlit app.

In [136]:
questions = [
    "How do I get camera inputs in a streamlit app ?",
    "without using a third party component ?",
    "Do you know the function camera_input() from streamlit ?",
]
chat_history = []

for question in questions:
    result = qa_over_streamlit_code({"question": question, "chat_history": chat_history})
    print(result)
    chat_history.append((question, result["answer"]))
    display(Markdown(result["answer"]))

{'question': 'How do I get camera inputs in a streamlit app ?', 'chat_history': [], 'answer': 'You can use the `streamlit-webrtc` library to get camera inputs in a Streamlit app. Here is an example of how you can use it:\n\n```python\nimport streamlit as st\nimport av\nimport streamlit_webrtc as webrtc\n\nwebrtc_ctx = webrtc.client(\n    key="camera-input",\n    video_transformer_factory=None,  # You can modify the video input if needed\n    async_transform=False,\n)\n\nvideo_transformer = webrtc_ctx.video_transformer()\n\nst.video(video_transformer, format="RGB")\n\n```\n\nFor more details, you can refer to the `streamlit-webrtc` documentation: https://docs.streamlit.io/en/stable/streamlit_webrtc.html'}


You can use the `streamlit-webrtc` library to get camera inputs in a Streamlit app. Here is an example of how you can use it:

```python
import streamlit as st
import av
import streamlit_webrtc as webrtc

webrtc_ctx = webrtc.client(
    key="camera-input",
    video_transformer_factory=None,  # You can modify the video input if needed
    async_transform=False,
)

video_transformer = webrtc_ctx.video_transformer()

st.video(video_transformer, format="RGB")

```

For more details, you can refer to the `streamlit-webrtc` documentation: https://docs.streamlit.io/en/stable/streamlit_webrtc.html

{'question': 'without using a third party component ?', 'chat_history': [('How do I get camera inputs in a streamlit app ?', 'You can use the `streamlit-webrtc` library to get camera inputs in a Streamlit app. Here is an example of how you can use it:\n\n```python\nimport streamlit as st\nimport av\nimport streamlit_webrtc as webrtc\n\nwebrtc_ctx = webrtc.client(\n    key="camera-input",\n    video_transformer_factory=None,  # You can modify the video input if needed\n    async_transform=False,\n)\n\nvideo_transformer = webrtc_ctx.video_transformer()\n\nst.video(video_transformer, format="RGB")\n\n```\n\nFor more details, you can refer to the `streamlit-webrtc` documentation: https://docs.streamlit.io/en/stable/streamlit_webrtc.html')], 'answer': 'No, it is not currently possible to directly get camera inputs in a Streamlit app without using a third-party component. However, you can use OpenCV or other libraries to capture and process camera inputs, and then display the processed video

No, it is not currently possible to directly get camera inputs in a Streamlit app without using a third-party component. However, you can use OpenCV or other libraries to capture and process camera inputs, and then display the processed video or images in your Streamlit app.

{'question': 'Do you know the function camera_input() from streamlit ?', 'chat_history': [('How do I get camera inputs in a streamlit app ?', 'You can use the `streamlit-webrtc` library to get camera inputs in a Streamlit app. Here is an example of how you can use it:\n\n```python\nimport streamlit as st\nimport av\nimport streamlit_webrtc as webrtc\n\nwebrtc_ctx = webrtc.client(\n    key="camera-input",\n    video_transformer_factory=None,  # You can modify the video input if needed\n    async_transform=False,\n)\n\nvideo_transformer = webrtc_ctx.video_transformer()\n\nst.video(video_transformer, format="RGB")\n\n```\n\nFor more details, you can refer to the `streamlit-webrtc` documentation: https://docs.streamlit.io/en/stable/streamlit_webrtc.html'), ('without using a third party component ?', 'No, it is not currently possible to directly get camera inputs in a Streamlit app without using a third-party component. However, you can use OpenCV or other libraries to capture and process c

Yes, the function `camera_input()` is available in the streamlit library. It is used to capture images from the user's camera.

In [77]:
questions = [
    "Do you know the function camera_input() from streamlit ?",
    "Give me an example of a streamlit app with a camera input",
]
chat_history = []

for question in questions:
    result = qa_over_streamlit_code({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: Do you know the function camera_input() from streamlit ? 

**Answer**: Yes, I am familiar with the function `camera_input()` from Streamlit. It is used to capture images from the user's camera. Here's an example usage:

```python
import streamlit as st

picture = st.camera_input("Take a picture")

if picture:
    st.image(picture)
```

This code will display a button labeled "Take a picture". When the button is clicked, the user's camera will open, allowing them to take a picture. The captured picture will then be displayed using the `st.image()` function. 

-> **Question**: Give me an example of a streamlit app with a camera input 

**Answer**: Certainly! Here's an example of a Streamlit app that uses the `camera_input()` function:

```python
import streamlit as st

def main():
    picture = st.camera_input("Take a picture")

    if picture:
        st.image(picture)

if __name__ == "__main__":
    main()
```

In this example, the `camera_input()` function is used to 

In [79]:
questions = [
    "What is the most recent version of streamlit ?",
    "What do you know about streamlit ?",
    "What do you know about langchain ?",
]
chat_history = []

for question in questions:
    result = qa_over_streamlit_code({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What is the most recent version of streamlit ? 

**Answer**: Based on the provided context, it is not possible to determine the most recent version of Streamlit. 

-> **Question**: What do you know about streamlit ? 

**Answer**: Streamlit is an open-source Python library used for building interactive web applications for data science and machine learning. It allows users to create and deploy data-driven apps quickly by simply writing Python scripts. With Streamlit, users can easily create and customize interactive visualizations, dashboards, and data exploration tools.

Streamlit provides a simple and intuitive API that allows users to add various UI components such as sliders, dropdowns, buttons, and plots to their apps. It also supports real-time data updates, allowing users to easily visualize and analyze changing data.

Some key features of Streamlit include:

- Easy-to-use API: Streamlit provides a concise and intuitive API, allowing users to build interactive ap

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 18105 tokens. Please reduce the length of the messages.

In [287]:
"""Chain for chatting with a vector database."""
from __future__ import annotations

import inspect
import warnings
from abc import abstractmethod
from pathlib import Path
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

from pydantic import Extra, Field, root_validator

from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
    Callbacks,
)
from langchain.chains.base import Chain
from langchain.chains.combine_documents.base import BaseCombineDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.base import BasePromptTemplate
from langchain.schema import BaseRetriever, Document

from langchain.chains.conversational_retrieval.base import CHAT_TURN_TYPE, _get_chat_history

from langchain.prompts.prompt import PromptTemplate

_template ="""You're an AI assistant specializing in python development. You know how to create Streamlit Applications.
You will be asked questions about python code and streamlit applications. You will be given a chat history and a python code for context.
Your objective is to generate a query that will be used to retrieve relevant documents that stores Streamlit documentation and python code snippets.
The query must be in a form of suite of words related to the context.

Python Code:
```python
{python_code}
```
Previous Questions:
{chat_history}
Follow Up Input: {question}
Query:"""

CONDENSE_QUESTION_CODE_PROMPT = PromptTemplate(template=_template, input_variables=["chat_history", "question", "python_code"])

class BaseConversationalRetrievalCodeChain(Chain):
    """Chain for chatting with an index. Given the chat history, the current code and a question, return the answer."""

    combine_docs_chain: BaseCombineDocumentsChain
    question_generator: LLMChain
    output_key: str = "answer"
    return_source_documents: bool = False
    return_generated_question: bool = False
    get_chat_history: Optional[Callable[[CHAT_TURN_TYPE], str]] = None
    """Return the source documents."""

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True
        allow_population_by_field_name = True

    @property
    def input_keys(self) -> List[str]:
        """Input keys."""
        return ["question", "chat_history", "python_code"]

    @property
    def output_keys(self) -> List[str]:
        """Return the output keys.

        :meta private:
        """
        _output_keys = [self.output_key]
        if self.return_source_documents:
            _output_keys = _output_keys + ["source_documents"]
        if self.return_generated_question:
            _output_keys = _output_keys + ["generated_question"]
        return _output_keys

    @abstractmethod
    def _get_docs(
        self,
        question: str,
        inputs: Dict[str, Any],
        *,
        run_manager: CallbackManagerForChainRun,
    ) -> List[Document]:
        """Get docs."""

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        question = inputs["question"]
        get_chat_history = self.get_chat_history or _get_chat_history
        chat_history_str = get_chat_history(inputs["chat_history"])
        source_code_str = inputs["python_code"]

        if chat_history_str or source_code_str:
            callbacks = _run_manager.get_child()
            new_question = self.question_generator.run(
                question=question, chat_history=chat_history_str, python_code=source_code_str, callbacks=callbacks
            )
        else:
            new_question = question
        accepts_run_manager = (
            "run_manager" in inspect.signature(self._get_docs).parameters
        )
        if accepts_run_manager:
            docs = self._get_docs(new_question, inputs, run_manager=_run_manager)
        else:
            docs = self._get_docs(new_question, inputs)  # type: ignore[call-arg]
        new_inputs = inputs.copy()
        # Remove "streamlit" or "python" from the question
        new_question = new_question.replace("streamlit", "").replace("python", "")
        print("new_question", new_question)
        new_inputs["chat_history"] = chat_history_str
        answer = self.combine_docs_chain.run(
            input_documents=docs, callbacks=_run_manager.get_child(), **new_inputs
        )
        output: Dict[str, Any] = {self.output_key: answer}
        if self.return_source_documents:
            output["source_documents"] = docs
        if self.return_generated_question:
            output["generated_question"] = new_question
        return output

    @abstractmethod
    async def _aget_docs(
        self,
        question: str,
        inputs: Dict[str, Any],
        *,
        run_manager: AsyncCallbackManagerForChainRun,
    ) -> List[Document]:
        """Get docs."""

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        _run_manager = run_manager or AsyncCallbackManagerForChainRun.get_noop_manager()
        question = inputs["question"]
        get_chat_history = self.get_chat_history or _get_chat_history
        chat_history_str = get_chat_history(inputs["chat_history"])
        source_code_str = inputs["python_code"]
        if chat_history_str:
            callbacks = _run_manager.get_child()
            new_question = await self.question_generator.arun(
                question=question, chat_history=chat_history_str, python_code=source_code_str, callbacks=callbacks
            )
        else:
            new_question = question
        accepts_run_manager = (
            "run_manager" in inspect.signature(self._aget_docs).parameters
        )
        if accepts_run_manager:
            docs = await self._aget_docs(new_question, inputs, run_manager=_run_manager)
        else:
            docs = await self._aget_docs(new_question, inputs)  # type: ignore[call-arg]

        new_inputs = inputs.copy()
        new_inputs["chat_history"] = chat_history_str
        answer = await self.combine_docs_chain.arun(
            input_documents=docs, callbacks=_run_manager.get_child(), **new_inputs
        )
        output: Dict[str, Any] = {self.output_key: answer}
        if self.return_source_documents:
            output["source_documents"] = docs
        if self.return_generated_question:
            output["generated_question"] = new_question
        return output

    def save(self, file_path: Union[Path, str]) -> None:
        if self.get_chat_history:
            raise ValueError("Chain not savable when `get_chat_history` is not None.")
        super().save(file_path)


class ConversationalRetrievalCodeChain(BaseConversationalRetrievalCodeChain):
    """Chain for chatting with an index."""

    retriever: BaseRetriever
    """Index to connect to."""
    max_tokens_limit: Optional[int] = None
    """If set, restricts the docs to return from store based on tokens, enforced only
    for StuffDocumentChain"""

    def _reduce_tokens_below_limit(self, docs: List[Document]) -> List[Document]:
        num_docs = len(docs)

        if self.max_tokens_limit and isinstance(
            self.combine_docs_chain, StuffDocumentsChain
        ):
            tokens = [
                self.combine_docs_chain.llm_chain.llm.get_num_tokens(doc.page_content)
                for doc in docs
            ]
            token_count = sum(tokens[:num_docs])
            while token_count > self.max_tokens_limit:
                num_docs -= 1
                token_count -= tokens[num_docs]

        return docs[:num_docs]

    def _get_docs(
        self,
        question: str,
        inputs: Dict[str, Any],
        *,
        run_manager: CallbackManagerForChainRun,
    ) -> List[Document]:
        """Get docs."""
        docs = self.retriever.get_relevant_documents(
            question, callbacks=run_manager.get_child()
        )
        return self._reduce_tokens_below_limit(docs)

    async def _aget_docs(
        self,
        question: str,
        inputs: Dict[str, Any],
        *,
        run_manager: AsyncCallbackManagerForChainRun,
    ) -> List[Document]:
        """Get docs."""
        docs = await self.retriever.aget_relevant_documents(
            question, callbacks=run_manager.get_child()
        )
        return self._reduce_tokens_below_limit(docs)

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        retriever: BaseRetriever,
        condense_question_prompt: BasePromptTemplate = CONDENSE_QUESTION_CODE_PROMPT,
        chain_type: str = "stuff",
        verbose: bool = False,
        condense_question_llm: Optional[BaseLanguageModel] = None,
        combine_docs_chain_kwargs: Optional[Dict] = None,
        callbacks: Callbacks = None,
        **kwargs: Any,
    ) -> BaseConversationalRetrievalCodeChain:
        """Load chain from LLM."""
        combine_docs_chain_kwargs = combine_docs_chain_kwargs or {}

        doc_chain = load_qa_chain(
            llm,
            chain_type=chain_type,
            verbose=verbose,
            callbacks=callbacks,
            **combine_docs_chain_kwargs,
        )

        _llm = condense_question_llm or llm
        condense_question_chain = LLMChain(
            llm=_llm,
            prompt=condense_question_prompt,
            verbose=verbose,
            callbacks=callbacks,
        )
        return cls(
            retriever=retriever,
            combine_docs_chain=doc_chain,
            question_generator=condense_question_chain,
            callbacks=callbacks,
            **kwargs,
        )

In [ ]:
def load_streamlit_doc_retriever():
    # Check if the Chroma database exists
    if not os.path.exists("../.doc_db/streamlit_chroma_db"):
        raise Exception("The Chroma database for Streamlit does not exist. Please run the script `doc_retriever.py` to create it.")

    # load from disk
    retriever = Chroma(persist_directory="../.doc_db/streamlit_chroma_db",
                       embedding_function=OpenAIEmbeddings(openai_api_key=openai_api_key)).as_retriever()
    retriever.search_kwargs["distance_metric"] = "cos"
    retriever.search_kwargs["fetch_k"] = 20
    retriever.search_kwargs["maximal_marginal_relevance"] = True
    retriever.search_kwargs["k"] = 4

    return retriever

In [261]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
model_name = "text-davinci-003"
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_api_key)  # 'ada' 'gpt-3.5-turbo' 'gpt-4',

retriever = streamlit_code_retriever
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 20
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 10

qa = ConversationalRetrievalCodeChain.from_llm(llm=llm, retriever=retriever, return_source_documents=True)

In [56]:
qa.run(question="How do I create a Streamlit app?", python_code="import streamlit as st\nst.write('Hello world!')")

"To create a Streamlit app, you can follow these steps:\n\n1. Install Streamlit by running the command `pip install streamlit` in your terminal.\n\n2. Create a new Python script and import the Streamlit library by adding the following line at the top of your script:\n   ```python\n   import streamlit as st\n   ```\n\n3. Write your app code using Streamlit's API. You can use various Streamlit functions like `st.write()`, `st.title()`, `st.markdown()`, etc., to display content in your app.\n\n4. Save your script with a `.py` extension, for example, `my_app.py`.\n\n5. Run your app by executing the command `streamlit run my_app.py` in your terminal. This will start a local server and open your app in a new tab in your browser.\n\n6. Modify your code, save it, and the changes will be automatically reflected in the browser. This allows you to see the updates live as you make changes to your app code.\n\nThat's it! You have created a Streamlit app. You can continue modifying your app code and

In [262]:
from IPython.display import display, Markdown
questions = [
    "I have a camera on my computer i want to display the real time image on the web page",
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history, "python_code":"import streamlit as st\nst.write('Hello world!')"})
    display(Markdown(result["answer"]))
    print(result["source_documents"])
    chat_history.append((question, result["answer"]))

new_question camera, real-time, display, image, web page


To display the real-time image from your computer's camera on a web page, you can use the browser's built-in capabilities to access the camera and display the video feed. This can be achieved using HTML5 and JavaScript.

Here is a basic example code:

```html
<!DOCTYPE html>
<html>
<head>
    <title>Camera Live Feed</title>
</head>
<body>
    <video id="camera-feed" autoplay></video>

    <script>
        // Access the camera feed
        navigator.mediaDevices.getUserMedia({ video: true })
            .then(function (stream) {
                var videoElement = document.getElementById('camera-feed');
                videoElement.srcObject = stream;
            })
            .catch(function (error) {
                console.log('Error accessing camera:', error);
            });
    </script>
</body>
</html>
```

In this example, the `getUserMedia()` function is used to access the camera feed. Once the camera access is granted, the `video` element with the id `camera-feed` is used to display the video stream.

Please note that camera access in the browser requires user permission, so the user will be prompted to grant permission to access the camera when they visit the web page.

[Document(page_content='Camera input live', metadata={'source': 'docs/content/library/api/widgets/widgets.md', 'filename': 'widgets.md', 'file_directory': 'docs/content/library/api/widgets', 'filetype': 'text/markdown', 'page_number': 2, 'category': 'Title'}), Document(page_content='Camera input live', metadata={'source': 'docs/content/library/api/api-reference.md', 'filename': 'api-reference.md', 'file_directory': 'docs/content/library/api', 'filetype': 'text/markdown', 'page_number': 2, 'category': 'Title'}), Document(page_content='Camera input', metadata={'source': 'docs/content/library/api/widgets/widgets.md', 'filename': 'widgets.md', 'file_directory': 'docs/content/library/api/widgets', 'filetype': 'text/markdown', 'page_number': 2, 'category': 'Title'}), Document(page_content='Camera input', metadata={'source': 'docs/content/library/api/api-reference.md', 'filename': 'api-reference.md', 'file_directory': 'docs/content/library/api', 'filetype': 'text/markdown', 'page_number': 2, 

In [113]:
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
    Callbacks,
)
qa_over_streamlit_code._get_docs(question="How do I get camera inputs in a streamlit app ?", inputs={}, run_manager=CallbackManagerForChainRun.get_noop_manager())

[Document(page_content='Examples\n        --------\n        >>> import streamlit as st\n        >>>\n        >>> picture = st.camera_input("Take a picture")\n        >>>\n        >>> if picture:\n        ...     st.image(picture)', metadata={'source': 'streamlit/lib/streamlit/elements/camera_input.py', 'content_type': 'functions_classes', 'language': 'python'}),
 Document(page_content='import streamlit as st\n\nx = st.camera_input("Label1", help="help1")\n\nif x is not None:\n    st.image(x)\n\ny = st.camera_input("Label2", help="help2", disabled=True)', metadata={'source': 'streamlit/e2e/scripts/st_camera_input.py', 'content_type': 'simplified_code', 'language': 'python'}),
 Document(page_content='import streamlit as st\n\nst.text_input("text_input")', metadata={'source': 'streamlit/e2e/scripts/app_hotkeys.py', 'content_type': 'simplified_code', 'language': 'python'}),
 Document(page_content='import streamlit as st\nfrom streamlit import runtime\n\ni1 = st.text_input("text input 1")\n

In [114]:
qa._get_docs(question="How do I get camera inputs in a streamlit app ?", inputs={}, run_manager=CallbackManagerForChainRun.get_noop_manager())

[Document(page_content='Examples\n        --------\n        >>> import streamlit as st\n        >>>\n        >>> picture = st.camera_input("Take a picture")\n        >>>\n        >>> if picture:\n        ...     st.image(picture)', metadata={'source': 'streamlit/lib/streamlit/elements/camera_input.py', 'content_type': 'functions_classes', 'language': 'python'}),
 Document(page_content='import streamlit as st\n\nx = st.camera_input("Label1", help="help1")\n\nif x is not None:\n    st.image(x)\n\ny = st.camera_input("Label2", help="help2", disabled=True)', metadata={'source': 'streamlit/e2e/scripts/st_camera_input.py', 'content_type': 'simplified_code', 'language': 'python'}),
 Document(page_content='import streamlit as st\n\nst.text_input("text_input")', metadata={'source': 'streamlit/e2e/scripts/app_hotkeys.py', 'content_type': 'simplified_code', 'language': 'python'}),
 Document(page_content='import streamlit as st\nfrom streamlit import runtime\n\ni1 = st.text_input("text input 1")\n

# Load up the Streamlit Doc

In [139]:

import os
import shutil
if os.path.exists("docs"):
    shutil.rmtree("docs")
os.system("git clone https://github.com/streamlit/docs.git -branch main --depth 1 docs")

Cloning into 'docs'...
Updating files: 100% (1110/1110), done.


0

In [292]:
from langchain.document_loaders import UnstructuredMarkdownLoader
steamlit_doc_loader = DirectoryLoader('docs', glob="content/**/*.md",
                                      loader_cls=UnstructuredMarkdownLoader,
                                      show_progress=True, use_multithreading=True)
steamlit_doc = steamlit_doc_loader.load()
steamlit_doc

  0%|          | 0/248 [00:00<?, ?it/s]

100%|██████████| 248/248 [00:27<00:00,  9.16it/s]


[Document(page_content='title: Welcome to Streamlit\nfeatures:\n  - title: Get started\n    body: Lorem ipsum dolor sit amet, consectetur adipiscing elit.\n    color: violet-70\n    icon: arrow_forward\n    url: /library/get-started\n    image: ""\n  - title: API reference\n    body: Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut\n      fugit, sed quia.\n    color: orange-70\n    icon: dvr\n    url: /library/api-reference\n    image: ""\n  - title: Topic guides\n    body: Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut\n      fugit, sed quia.\n    color: l-blue-70\n    icon: description\n    url:\n      Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit,\n      sed quia.\n    image: ""\nwhats_new:\n  - title: Feature title\n    body: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Eleifend a\n      facilisis sagittis, vitae nibh massa in facilisis et. Pretium eget non\n      cursus purus tempus porta sodales.\n

In [299]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 2000,
    chunk_overlap  = 50,
)
steamlit_doc_splitted = text_splitter.split_documents(steamlit_doc)
steamlit_doc_splitted

[Document(page_content='title: Welcome to Streamlit\nfeatures:\n  - title: Get started\n    body: Lorem ipsum dolor sit amet, consectetur adipiscing elit.\n    color: violet-70\n    icon: arrow_forward\n    url: /library/get-started\n    image: ""\n  - title: API reference\n    body: Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut\n      fugit, sed quia.\n    color: orange-70\n    icon: dvr\n    url: /library/api-reference\n    image: ""\n  - title: Topic guides\n    body: Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut\n      fugit, sed quia.\n    color: l-blue-70\n    icon: description\n    url:\n      Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit,\n      sed quia.\n    image: ""\nwhats_new:\n  - title: Feature title\n    body: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Eleifend a\n      facilisis sagittis, vitae nibh massa in facilisis et. Pretium eget non\n      cursus purus tempus porta sodales.\n

In [157]:
loader = GenericLoader.from_filesystem(
    "docs/",
    glob="python/api-examples-source/**/*.py",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON),
    show_progress=True
)
streamlit_code_example_doc = loader.load()

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=500, chunk_overlap=50
)
streamlit_code_example_doc_splitted = python_splitter.split_documents(streamlit_code_example_doc)

 20%|█▉        | 22/112 [00:00<00:00, 217.85it/s]

100%|██████████| 112/112 [00:00<00:00, 185.52it/s]


In [300]:
token, max_token = num_tokens_from_string(steamlit_doc_splitted, "cl100k_base")
print(f"Number of tokens in source code: {token}, max tokens in source code: {max_token}, price: {token * 0.0001/1000}")
token, max_token = num_tokens_from_string(streamlit_code_example_doc_splitted, "cl100k_base")
print(f"Number of tokens in source code: {token}, max tokens in source code: {max_token}, price: {token * 0.0001/1000}")

Number of tokens in source code: 163803, max tokens in source code: 582, price: 0.0163803
Number of tokens in source code: 23121, max tokens in source code: 161, price: 0.0023121


In [301]:
embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore_code = Chroma.from_documents(documents=steamlit_doc_splitted + streamlit_code_example_doc_splitted,embedding=embeddings_model)

In [302]:
streamlit_code_retriever = vectorstore_code.as_retriever()
streamlit_code_retriever.search_kwargs["distance_metric"] = "cos"
streamlit_code_retriever.search_kwargs["fetch_k"] = 20
streamlit_code_retriever.search_kwargs["maximal_marginal_relevance"] = True
streamlit_code_retriever.search_kwargs["k"] = 20

In [311]:
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
from langchain.memory import ConversationBufferMemory
model_name = "gpt-3.5-turbo"
model_name = "text-davinci-003"
llm = OpenAI(temperature=0, model_name=model_name, openai_api_key=openai_api_key)
#llm = ChatOpenAI(temperature=0, model_name=model_name, openai_api_key=openai_api_key)  # 'ada' 'gpt-3.5-turbo' 'gpt-4',

retriever = streamlit_code_retriever
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 5
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 5


prompt_template = """You're an AI assistant specializing in python development. 
You will be given a question, the current python code to modify with and several documents.
Your goal is to modify the python code to answer the question. You can use the documents to get up to date Streamlit api references.
Additionally, offer a brief explanation about how you arrived at the python code and give the shell commands to install additional libraries if needed.
If the input is a question, you must explain the code only and additionnaly propose some code. Do not halucinate or make up information. If you do not know the answer, just say "I don't know".

Documents related to the question:
{context}

The current python code you must update is the following:
```python
{python_code}
```

Write your anwser in the following format:
```python
the code you generated
```
the explanation of the code you generated

If you did not generated any code (for instance when the user ask a question, not an instruction), this is the format:
```python
None
```
the anwser to the question

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["python_code", "context", "question"]
)
combine_docs_chain_kwargs = {"prompt": PROMPT}

qa_over_streamlit_code = ConversationalRetrievalCodeChain.from_llm(llm=llm, retriever=retriever, return_source_documents=True, combine_docs_chain_kwargs=combine_docs_chain_kwargs)

In [319]:
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
    Callbacks,
)
qa_over_streamlit_code._get_docs(question="display camera input", inputs={}, run_manager=CallbackManagerForChainRun.get_noop_manager())

[Document(page_content='webcam_demo()\n\nshow_code(webcam_demo)', metadata={'source': 'docs/python/api-examples-source/mpa-hello/pages/4_📷_Webcam_Demo.py', 'content_type': 'simplified_code', 'language': 'python'}),
 Document(page_content="title: Enabling camera access in your browser\nslug: /knowledge-base/using-streamlit/enable-camera\n\nEnabling camera access in your browser\n\nStreamlit apps may include a widget to upload images directly from your computer's camera. To\nsafeguard the users' privacy and security, browsers require users to explicitly allow access to the\ncamera before this widget can work.\n\nTo learn how to enable camera access, please check the documentation for your browser:\n\nChrome\n\nSafari\n\nFirefox", metadata={'source': 'docs/content/kb/using-streamlit/enable-camera.md'}),
 Document(page_content='title: st.camera_input\nslug: /library/api-reference/widgets/st.camera_input\ndescription: st.camera_input displays a widget to upload images from a camera\n\nTo re

In [314]:
import re
def parse(output):
    python_code = None
    explain_code = None
    pattern = r"```python\n(.*?)\n```.*\n(.*?)$"
    python_code_match = re.search(pattern, output, re.DOTALL | re.MULTILINE)
    if python_code_match:
        python_code = python_code_match.group(1)
        explain_code = python_code_match.group(2)
        if python_code == "None":
            python_code = None
    return python_code, explain_code

In [1]:
from IPython.display import display, Markdown
questions = [
    "Add a title to the page",
    "change the title text to 'Hello world!'",
    "Get the input from a camera and display it",
]
last_code = "import streamlit as st\nst.write('Hello world!')"
chat_history = []

for question in questions:
    result = qa_over_streamlit_code({"question": question, "chat_history": chat_history, "python_code": last_code})
    print(result)
    display(Markdown(result["answer"]))
    code, explain = parse(result["answer"])
    last_code = code
    chat_history.append((question, explain))

NameError: name 'qa_over_streamlit_code' is not defined

In [306]:
from IPython.display import display, Markdown
questions = [
    "But can you modify the app ?",
]

for question in questions:
    result = qa_over_streamlit_code({"question": question, "chat_history": chat_history, "python_code": last_code})
    print(result)
    display(Markdown(result["answer"]))
    code, explain = parse(result["answer"])
    last_code = code
    chat_history.append((question, explain))

new_question  " camera_input Image module error fix"
{'question': "I have this error 'no module named Image', fix it", 'chat_history': [('Add a title to the page', 'This code will add a title to the page using the Streamlit built-in st.title() function.'), ("change the title text to 'Hello world!'", "This code will change the title text to 'Hello world!' using the Streamlit built-in st.title() function."), ('Get the input from a camera and display it', 'This code will get the input from the camera and display it on the screen using the Streamlit built-in st.camera_input() and st.image() functions.')], 'python_code': 'import streamlit as st\n\nimg_file_buffer = st.camera_input("Take a picture")\n\nif img_file_buffer is not None:\n    # To read image file buffer as a PIL Image:\n    img = Image.open(img_file_buffer)\n    st.image(img, caption=\'Image from camera\', use_column_width=True)', 'answer': '\n```python\nNone\n```\nThis error occurs when you try to import a module that is not in


```python
None
```
This error occurs when you try to import a module that is not included in your requirements file. To fix this, you need to install the module in your requirements file. For example, if you are trying to import the Image module, you need to add the following line to your requirements file:

Pillow==7.2.0

In [14]:
from langchain import OpenAI, ConversationChain, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI


template = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

{history}
Human: {human_input}
Assistant:"""

prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)


chatgpt_chain = LLMChain(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo-16k", openai_api_key=openai_api_key),
    prompt=prompt,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=2),
)

output = chatgpt_chain.predict(
    human_input="What is the version of streamlit ?",
)
print(output)



> Entering new  chain...
Prompt after formatting:
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a pow


> Finished chain.
The current version of Streamlit, as of September 2021, is 0.89.0. However, it's important to note that software versions can change over time as new updates and releases are made. It's always a good idea to check the official Streamlit website or documentation for the most up-to-date information on the current version.
